In [18]:
from keras.datasets import cifar10
from model import resnet

from cleverhans.attacks import FastGradientMethod
from cleverhans.attacks import ProjectedGradientDescent
from cleverhans.compat import flags
from cleverhans.dataset import MNIST
from cleverhans.utils import AccuracyReport
from cleverhans.utils_keras import cnn_model
from cleverhans.utils_keras import KerasModelWrapper

import tensorflow as tf
from tensorflow import keras
from keras.optimizers import adam
import numpy as np


In [2]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

In [3]:
input_shape = x_train.shape[1:]
print(x_train.shape)
model = resnet(input_shape=(32,32,3), n_classes=10)
model.summary()

(50000, 32, 32, 3)
Instructions for updating:
Colocations handled automatically by placer.
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 32, 32, 3)    0                                            
__________________________________________________________________________________________________
zero_padding2d (ZeroPadding2D)  (None, 38, 38, 3)    0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 16, 16, 64)   9472        zero_padding2d[0][0]             
__________________________________________________________________________________________________
bn_conv1 (BatchNormalizationV1) (None, 16, 16, 64)   256         conv1[0][0]                      
__________________

In [4]:
# Object used to keep track of (and return) key accuracies
report = AccuracyReport()

# Set TF random seed to improve reproducibility
tf.set_random_seed(1234)
# Force TensorFlow to use single thread to improve reproducibility
config = tf.ConfigProto(intra_op_parallelism_threads=1,
                      inter_op_parallelism_threads=1)

if keras.backend.image_data_format() != 'channels_last':
    raise NotImplementedError("this tutorial requires keras to be configured to channels_last format")

# Create TF session and set as Keras backend session
sess = tf.Session(config=config)
keras.backend.set_session(sess)

img_rows, img_cols, nchannels = x_train.shape[1:4]
nb_classes = y_train.shape[1]
# y_train -= label_smoothing * (y_train - 1. / nb_classes)

model(model.get_input_at(0))

<tf.Tensor 'ResNet50/output10/Softmax:0' shape=(?, 10) dtype=float32>

In [5]:
def get_adversarial_acc_metric(model, fgsm, fgsm_params):
    print("in metric")
    def adv_acc(y, _):
        # Generate adversarial examples
        x_adv = fgsm.generate(model.get_input_at(0), **fgsm_params)
        # Consider the attack to be constant
        x_adv = tf.stop_gradient(x_adv)

        # Accuracy on the adversarial examples
        preds_adv = model(x_adv)
        return keras.metrics.categorical_accuracy(y, preds_adv)

    return adv_acc


def get_adversarial_loss(model, fgsm, fgsm_params):
    print("in here")
    def adv_loss(y, preds):
        # Cross-entropy on the legitimate examples
        cross_ent = keras.losses.categorical_crossentropy(y, preds)

        # Generate adversarial examples
        x_adv = fgsm.generate(model.get_input_at(0), **fgsm_params)
        # Consider the attack to be constant
        x_adv = tf.stop_gradient(x_adv)

        # Cross-entropy on the adversarial examples
        preds_adv = model(x_adv)
        cross_ent_adv = keras.losses.categorical_crossentropy(y, preds_adv)

        return 0.5 * cross_ent + 0.5 * cross_ent_adv

    return adv_loss



In [9]:
wrap_2 = KerasModelWrapper(model)

# fgsm_2 = FastGradientMethod(wrap_2, sess=sess)
fgsm_2 = ProjectedGradientDescent(wrap_2, sess=sess)

fgsm_params = {'eps': 0.3,
             'clip_min': 0.,
             'clip_max': 1.}

learning_rate = 0.001
opt = adam(lr=0.001, decay=1e-6)

# Use a loss function based on legitimate and adversarial examples
adv_loss_2 = get_adversarial_loss(model, fgsm_2, fgsm_params)
adv_acc_metric_2 = get_adversarial_acc_metric(model, fgsm_2, fgsm_params)
model.compile(
  optimizer=keras.optimizers.Adam(learning_rate),
  loss=adv_loss_2,
  metrics=['accuracy', adv_acc_metric_2]
)

/home/xun/anaconda3/lib/python3.6/site-packages/cleverhans/attacks_tf.py:27: UserWarning: attacks_tf is deprecated and will be removed on 2019-07-18 or after. Code should import functions from their new locations directly.
  warnings.warn("attacks_tf is deprecated and will be removed on 2019-07-18"


in here
in metric
Instructions for updating:
Use tf.cast instead.
Instructions for updating:
dim is deprecated, use axis instead


/home/xun/anaconda3/lib/python3.6/site-packages/cleverhans/compat.py:22: UserWarning: <function reduce_sum_v1 at 0x7faf38075378> is deprecated. Switch to calling the equivalent function in tensorflow.  This function was originally needed as a compatibility layer for old versions of tensorflow,  but support for those versions has now been dropped.
  warnings.warn(str(f) + " is deprecated. Switch to calling the equivalent function in tensorflow. "


Instructions for updating:
Use tf.cast instead.


In [25]:
fgsm_2.generate(x_train[:1].astype(int))

TypeError: Expected int64 passed to parameter 'y' of op 'Minimum', got 0.3 of type 'float' instead.

In [ ]:
# Train an MNIST model
model.fit(x_train[:1000], y_train[:1000],
          batch_size=1000,
          epochs=1,
          validation_data=(x_test[:10], y_test[:10]),
          verbose=1)

# Evaluate the accuracy on legitimate and adversarial test examples
_, acc, adv_acc = model.evaluate(x_test, y_test,
                                 batch_size=batch_size,
                                 verbose=1)
report.adv_train_clean_eval = acc
report.adv_train_adv_eval = adv_acc
print('Test accuracy on legitimate examples: %0.4f' % acc)
print('Test accuracy on adversarial examples: %0.4f\n' % adv_acc)

# # Calculate training error
# if testing:
#     _, train_acc, train_adv_acc = model_2.evaluate(x_train, y_train,
#                                                batch_size=batch_size,
#                                                verbose=0)
# report.train_adv_train_clean_eval = train_acc
# report.train_adv_train_adv_eval = train_adv_acc

Train on 1000 samples, validate on 10 samples


In [ ]:
print("tesT")